#Data Loading

In [0]:
import pandas as pd
import numpy as np
import datetime as dt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.preprocessing import MinMaxScaler

In [0]:
dev=pd.read_csv('/content/drive/My Drive/Project Future/Project/data/devices.csv')
notif=pd.read_csv('/content/drive/My Drive/Project Future/Project/data/notifications.csv')
tr1=pd.read_csv('/content/drive/My Drive/Project Future/Project/data/transactions_1.csv')
tr2=pd.read_csv('/content/drive/My Drive/Project Future/Project/data/transactions_2.csv')
tr3=pd.read_csv('/content/drive/My Drive/Project Future/Project/data/transactions_3.csv')
us=pd.read_csv('/content/drive/My Drive/Project Future/Project/data/users.csv')

In [0]:
tr=pd.DataFrame()
tr=tr.append(tr1)
tr=tr.append(tr2)
tr=tr.append(tr3)

#Transactions

In [0]:
trans=tr
trans.columns

Index(['transaction_id', 'transactions_type', 'transactions_currency',
       'amount_usd', 'transactions_state', 'ea_cardholderpresence',
       'ea_merchant_mcc', 'ea_merchant_city', 'ea_merchant_country',
       'direction', 'user_id', 'created_date'],
      dtype='object')

In [0]:
trans=trans.drop(columns=["transaction_id","transactions_currency","ea_cardholderpresence","ea_merchant_city","ea_merchant_country"])
trans.columns

Index(['transactions_type', 'amount_usd', 'transactions_state',
       'ea_merchant_mcc', 'direction', 'user_id', 'created_date'],
      dtype='object')

In [0]:
print(trans.transactions_state.value_counts(),"\n")
print(trans.transactions_type.value_counts(),"\n")
print(trans.direction.value_counts(),"\n")

COMPLETED    1916387
DECLINED      124226
REVERTED       89330
FAILED         32566
PENDING        17153
CANCELLED       1364
Name: transactions_state, dtype: int64 

CARD_PAYMENT    1173661
TRANSFER         402126
TOPUP            309740
EXCHANGE         124800
ATM               74181
CASHBACK          65063
FEE               18786
CARD_REFUND        9209
TAX                2267
REFUND             1193
Name: transactions_type, dtype: int64 

OUTBOUND    1759079
INBOUND      421947
Name: direction, dtype: int64 



In [0]:
print("All rows ",trans.shape[0])
trans=trans[trans.amount_usd<=2.5*10**6]
print("amount_usd<2.5*10^6 rows: ",trans.shape[0])
trans=trans[(trans.transactions_state=="COMPLETED")]
print("Only completed trans: ",trans.shape[0])
trans.describe()

All rows  2181026
amount_usd<2.5*10^6 rows:  2180930
Only completed trans:  1916387


,amount_usd,ea_merchant_mcc
count,1.916387e+06,1.070144e+06
mean,6.452131e+01,5.695772e+03
std,9.122444e+02,9.157216e+02
min,0.000000e+00,7.420000e+02
25%,2.400000e+00,5.411000e+03
50%,8.510000e+00,5.812000e+03
75%,2.809000e+01,5.815000e+03
max,6.127665e+05,9.406000e+03


In [0]:
#Κρατάμε outbound Transfer, Exchange, card_payment, ATM και Inbound Transfer, TopUp
test=trans.pivot_table(columns="direction",index=["transactions_type","transactions_state"],\
                       values="amount_usd",aggfunc="sum",fill_value="",margins="row grand total")
test

,direction,INBOUND,OUTBOUND,All
transactions_type,transactions_state,,,
ATM,COMPLETED,,4.34584e+06,4.34584e+06
CARD_PAYMENT,COMPLETED,,2.0403e+07,2.0403e+07
CARD_REFUND,COMPLETED,598569,,598569
CASHBACK,COMPLETED,5561.8,,5561.8
EXCHANGE,COMPLETED,,2.4137e+07,2.4137e+07
FEE,COMPLETED,,152523,152523
REFUND,COMPLETED,477460,,477460
TAX,COMPLETED,,224.18,224.18
TOPUP,COMPLETED,4.62064e+07,,4.62064e+07


In [0]:
trans=trans[(trans.transactions_type=="ATM")|(trans.transactions_type=="EXCHANGE")|(trans.transactions_type=="TOPUP")|(trans.transactions_type=="CARD_PAYMENT")|(trans.transactions_type=="TRANSFER")]
print("useful rows: ",trans.shape[0])
test=trans.pivot_table(columns="direction",index=["transactions_type","transactions_state"],values="amount_usd",aggfunc="count",fill_value="",margins="row grand total")
test

useful rows:  1824680


,direction,INBOUND,OUTBOUND,All
transactions_type,transactions_state,,,
ATM,COMPLETED,,61518,61518
CARD_PAYMENT,COMPLETED,,999418,999418
EXCHANGE,COMPLETED,,124800,124800
TOPUP,COMPLETED,242621,,242621
TRANSFER,COMPLETED,36169,360154,396323
All,,278790,1545890,1824680


##Transaction statistics

In [0]:
test=trans[(trans.transactions_type=="ATM")|(trans.transactions_type=="EXCHANGE")|(trans.transactions_type=="CARD_PAYMENT")|(trans.transactions_type=="TRANSFER")|(trans.direction=="OUTBOUND")]
trans_stat=test.pivot_table(index="user_id",values="amount_usd",aggfunc=["min","max","mean"],fill_value=0)
trans_stat=pd.DataFrame(trans_stat.to_records())
trans_stat=trans_stat.rename(columns={"('min', 'amount_usd')":"min_amount_usd","('max', 'amount_usd')":"max_amount_usd","('mean', 'amount_usd')":"mean_amount_usd"})
trans_stat

,user_id,min_amount_usd,max_amount_usd,mean_amount_usd
0,user_0,0.58,510.64,10.209723
1,user_1,0.94,851.06,61.938624
2,user_10,0.01,255.22,37.871972
3,user_100,2.03,339.18,47.975909
4,user_1000,0.08,158.87,21.760000
...,...,...,...,...
13846,user_9994,1.09,372.81,38.523548
13847,user_9995,51.71,82.96,62.453333
13848,user_9996,2.12,166.81,38.337308
13849,user_9997,0.80,1000.00,45.526458


##Ανα mcc και τύπο Trans

In [0]:
trans_type = trans.pivot_table(values="amount_usd",index="user_id",columns=["transactions_type","direction"],aggfunc="sum",fill_value=0)
trans_type = pd.DataFrame(trans_type.to_records())
trans_type = trans_type.rename(columns={"('ATM', 'OUTBOUND')":"ATM","('CARD_PAYMENT', 'OUTBOUND')":"CARD_PAYMENT","('EXCHANGE', 'OUTBOUND')":"EXCHANGE",\
                                       "('TRANSFER', 'INBOUND')":"TRANSFER_IN","('TRANSFER', 'OUTBOUND')":"TRANSFER_OUT","('All', '')":"TOTAL",\
                                       "('TOPUP', 'INBOUND')":"TOPUP_IN"})
trans_type["total_out"]=trans_type.ATM+trans_type.CARD_PAYMENT+trans_type.TRANSFER_OUT+trans_type.EXCHANGE
trans_type["total_in"]=trans_type.TOPUP_IN+trans_type.TRANSFER_IN

trans_type=trans_type.drop(columns=["CARD_PAYMENT","ATM"])

trans_type.head(10)

,user_id,EXCHANGE,TOPUP_IN,TRANSFER_IN,TRANSFER_OUT,total_out,total_in
0,user_0,183.20,3226.37,55.96,1246.62,3629.75,3282.33
1,user_1,0.00,6204.23,261.28,2865.53,6490.03,6465.51
2,user_10,1.22,2264.05,0.00,803.07,2688.91,2264.05
3,user_100,0.00,2119.16,0.00,0.00,2110.94,2119.16
4,user_1000,0.00,428.96,0.00,0.08,391.68,428.96
5,user_10000,922.15,1261.35,342.79,312.88,2445.59,1604.14
6,user_10001,1120.00,1120.00,0.00,0.00,2226.32,1120.00
7,user_10002,3.92,13.98,0.00,0.00,11.12,13.98
8,user_10003,0.00,1604.79,0.00,462.16,1590.15,1604.79
9,user_10004,23.83,8.51,0.00,0.00,23.83,8.51


In [0]:
#MCC clustering
trans.ea_merchant_mcc=trans.ea_merchant_mcc.fillna(11999)
 
trans['mcc_group'] = pd.cut(trans.ea_merchant_mcc, bins=[0,1000,1500,1800,2000,4000,5000,5200,6000,6800,9000,9730,10000,12000],\
                                  labels=['Agriculture','Mining','Construction',"not used","Manufacturing","Transportation","Wholesale",\
                                          "Retail","Finance","Services","Public_Admin","Nonclassifiable","Unknown"], right=False)
test=trans.pivot_table(index="mcc_group",values="user_id",aggfunc="count",margins="row grand total")
test=pd.DataFrame(test.to_records())
test.sort_values("user_id",ascending=False)

,mcc_group,user_id
13,All,1824680
12,Unknown,763744
7,Retail,760162
5,Transportation,119706
9,Services,95847
8,Finance,63692
4,Manufacturing,8565
10,Public_Admin,5942
6,Wholesale,4176
0,Agriculture,1285


In [0]:
trans_mcc_all = trans.pivot_table(values="amount_usd",index="user_id",columns="mcc_group",aggfunc="sum",fill_value=0)
trans_mcc_all = pd.DataFrame(trans_mcc_all.to_records())
trans_mcc_all=trans_mcc_all.drop(columns="Unknown")
trans_mcc_all.sample(5)

,user_id,Agriculture,Mining,Construction,Manufacturing,Transportation,Wholesale,Retail,Finance,Services,Public_Admin
14109,user_9355,0.0,0.0,0.0,341.89,998.70,4.48,6480.39,1865.41,806.57,0.0
2529,user_12386,0.0,0.0,0.0,195.27,167.67,0.00,1902.92,230.00,124.01,0.0
1818,user_11711,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0
4722,user_14448,0.0,0.0,0.0,0.00,0.00,0.00,1.98,0.00,0.00,0.0
6604,user_2239,0.0,0.0,0.0,0.00,198.52,0.00,744.17,356.62,1498.24,0.0


In [0]:
print(trans_mcc_all.shape)
trans_type.shape

(14796, 11)


(14796, 7)

In [0]:
trans_merge=pd.merge(trans_mcc_all,trans_type,on="user_id",how="inner")
print(trans_merge.shape)
trans_merge.head()



(14796, 17)


,user_id,Agriculture,Mining,Construction,Manufacturing,Transportation,Wholesale,Retail,Finance,Services,Public_Admin,EXCHANGE,TOPUP_IN,TRANSFER_IN,TRANSFER_OUT,total_out,total_in
0,user_0,0.0,0.0,0.0,0.00,161.73,0.00,2019.30,0.00,18.90,0.00,183.20,3226.37,55.96,1246.62,3629.75,3282.33
1,user_1,0.0,0.0,0.0,0.00,58.51,0.00,1565.94,1183.02,759.00,58.03,0.00,6204.23,261.28,2865.53,6490.03,6465.51
2,user_10,0.0,0.0,0.0,255.22,0.00,190.66,1019.18,408.51,11.05,0.00,1.22,2264.05,0.00,803.07,2688.91,2264.05
3,user_100,0.0,0.0,0.0,119.51,60.29,40.58,1036.96,425.66,427.94,0.00,0.00,2119.16,0.00,0.00,2110.94,2119.16
4,user_1000,0.0,0.0,0.0,0.00,10.80,0.00,210.01,158.87,11.92,0.00,0.00,428.96,0.00,0.08,391.68,428.96


In [0]:
trans_merge.Agriculture = round(trans_merge.Agriculture/trans_merge.total_out,4)*100
trans_merge.Mining = round(trans_merge.Mining/trans_merge.total_out,4)*100
trans_merge.Construction = round(trans_merge.Construction/trans_merge.total_out,4)*100
trans_merge.Manufacturing = round(trans_merge.Manufacturing/trans_merge.total_out,4)*100
trans_merge.Transportation = round(trans_merge.Transportation/trans_merge.total_out,4)*100
trans_merge.Wholesale = round(trans_merge.Wholesale/trans_merge.total_out,4)*100
trans_merge.Retail = round(trans_merge.Retail/trans_merge.total_out,4)*100
trans_merge.Finance = round(trans_merge.Finance/trans_merge.total_out,4)*100
trans_merge.Services = round(trans_merge.Services/trans_merge.total_out,4)*100
trans_merge.Public_Admin = round(trans_merge.Public_Admin/trans_merge.total_out,4)*100

trans_merge.EXCHANGE = round(trans_merge.EXCHANGE/trans_merge.total_out,4)*100
trans_merge.TRANSFER_IN = round(trans_merge.TRANSFER_IN/trans_merge.total_in,4)*100
trans_merge.TRANSFER_OUT = round(trans_merge.TRANSFER_OUT/trans_merge.total_out,4)*100
trans_merge.TOPUP_IN = round(trans_merge.TOPUP_IN/trans_merge.total_in,4)*100

trans_merge.sample(5)

,user_id,Agriculture,Mining,Construction,Manufacturing,Transportation,Wholesale,Retail,Finance,Services,Public_Admin,EXCHANGE,TOPUP_IN,TRANSFER_IN,TRANSFER_OUT,total_out,total_in
6444,user_2090,0.0,0.0,0.0,0.0,1.98,0.0,53.42,7.23,0.00,0.0,10.86,59.35,40.65,26.50,239.44,292.77
13515,user_8786,0.0,0.0,0.0,0.0,0.00,0.0,1.32,1.93,0.10,0.0,48.91,100.00,0.00,47.75,35327.09,17774.68
7358,user_2960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.00,0.00,NaN,0.00,20.00
5545,user_15222,0.0,0.0,0.0,0.0,0.00,0.0,23.98,0.00,11.02,0.0,65.00,100.00,0.00,0.00,893.18,320.00
10490,user_5947,0.0,0.0,0.0,0.0,5.82,0.0,9.89,2.63,7.92,0.0,33.75,100.00,0.00,39.99,1355.59,918.76


#Notifications File

In [0]:
ntf=notif

In [0]:
ntf.columns

Index(['reason', 'channel', 'status', 'user_id', 'created_date'], dtype='object')

In [0]:
test=ntf.pivot_table(index="reason",columns="channel",values="user_id",aggfunc="count",fill_value="",\
                     margins="row total").drop(columns=["All"])
test

channel,EMAIL,PUSH,SMS
reason,,,
BLACK_FRIDAY,,1681,
BLUE_TUESDAY,,555,
ENGAGEMENT_SPLIT_BILL_RESTAURANT,1834,,
INVEST_IN_GOLD,6649,,
JOINING_ANNIVERSARY,,3334,
LOST_CARD_ORDER,1888,3053,
MADE_MONEY_REQUEST_NOT_SPLIT_BILL,2851,,
METAL_RESERVE_PLAN,,346,604
NO_INITIAL_CARD_ORDER,9071,,


#Devices File

In [0]:
dev.columns

Index(['brand', 'user_id'], dtype='object')

#Users File

In [0]:
users=us
users.columns

Index(['user_id', 'birth_year', 'country', 'city', 'created_date',
       'user_settings_crypto_unlocked', 'plan',
       'attributes_notifications_marketing_push',
       'attributes_notifications_marketing_email', 'num_contacts',
       'num_referrals', 'num_successful_referrals'],
      dtype='object')

In [0]:
users=users.drop(columns=["city",'num_referrals', 'num_successful_referrals',\
                          'attributes_notifications_marketing_push','attributes_notifications_marketing_email'])
users.columns

Index(['user_id', 'birth_year', 'country', 'created_date',
       'user_settings_crypto_unlocked', 'plan', 'num_contacts'],
      dtype='object')

In [0]:
users["user_age"] =  dt.date.today().year-users['birth_year']

users=users.drop(columns=["created_date","birth_year"])
users.sample(5)

,user_id,country,user_settings_crypto_unlocked,plan,num_contacts,user_age
14664,user_14664,PL,1,STANDARD,4,36
11932,user_11932,FR,0,STANDARD,1,30
4376,user_4376,LV,0,STANDARD,52,45
5108,user_5108,IE,0,STANDARD,0,64
12466,user_12466,FR,0,STANDARD,0,73


In [0]:
#Countries clustering
countries_cluster=pd.DataFrame({"Cluster":["CEE_EU"]*8+["N_EU"]*8+["S_EU"]*7+["W_EU"]*13+["Non_EU"]*4,"country":\
                                ['BG','RO','PL','CZ','SI','HU','HR','SK',\
                                'LT','DK','EE','FI','IS','LV','NO','SE',\
                                'ES','MT','CY','GR','IT','GI','PT',\
                                'AT','IE','GB','BE','CH','DE','GG','IM','JE','LI','LU','NL',"FR",\
                                'AU','GP','MQ','RE']})
countries_cluster

users=pd.merge(users,countries_cluster,how="left",on="country")
users=users.drop(columns="country")
users.head()

,user_id,user_settings_crypto_unlocked,plan,num_contacts,user_age,Cluster
0,user_0,0,STANDARD,3,28,S_EU
1,user_1,1,STANDARD,75,35,N_EU
2,user_2,0,STANDARD,71,24,W_EU
3,user_3,0,STANDARD,219,39,W_EU
4,user_4,0,STANDARD,0,28,W_EU


In [0]:
users.loc[users['plan']== "STANDARD", 'paid_plan'] = 0 
users.loc[users['plan']!= "STANDARD", 'paid_plan'] = 1 
users=users.drop(columns="plan")
users.sample(5)

,user_id,user_settings_crypto_unlocked,num_contacts,user_age,Cluster,paid_plan
7669,user_7669,0,15,22,N_EU,0.0
3059,user_3059,0,20,27,CEE_EU,0.0
4927,user_4927,1,11,50,W_EU,0.0
9223,user_9223,0,0,55,W_EU,0.0
1202,user_1202,0,0,26,W_EU,0.0


In [0]:
users=users.rename(columns={"user_settings_crypto_unlocked":"crypto_on"})
users.sample(5)

,user_id,crypto_on,num_contacts,user_age,Cluster,paid_plan
12943,user_12943,0,44,36,N_EU,0.0
12277,user_12277,0,19,39,CEE_EU,0.0
14739,user_14739,0,0,20,W_EU,0.0
10239,user_10239,0,1,54,W_EU,0.0
1555,user_1555,0,6,38,W_EU,0.0


#User Modelling File

##File Creation

In [0]:
users_mod=pd.merge(left=users,right=trans_merge,on="user_id",how="left")
users_mod=pd.merge(left=users_mod,right=dev,on="user_id",how="left")
users_mod=pd.merge(left=users_mod,right=trans_stat,on="user_id",how="left")
print(users_mod.columns)
users_mod.sample(5)

Index(['user_id', 'crypto_on', 'num_contacts', 'user_age', 'Cluster',
       'paid_plan', 'Agriculture', 'Mining', 'Construction', 'Manufacturing',
       'Transportation', 'Wholesale', 'Retail', 'Finance', 'Services',
       'Public_Admin', 'EXCHANGE', 'TOPUP_IN', 'TRANSFER_IN', 'TRANSFER_OUT',
       'total_out', 'total_in', 'brand', 'min_amount_usd', 'max_amount_usd',
       'mean_amount_usd'],
      dtype='object')


,user_id,crypto_on,num_contacts,user_age,Cluster,paid_plan,Agriculture,Mining,Construction,Manufacturing,Transportation,Wholesale,Retail,Finance,Services,Public_Admin,EXCHANGE,TOPUP_IN,TRANSFER_IN,TRANSFER_OUT,total_out,total_in,brand,min_amount_usd,max_amount_usd,mean_amount_usd
8778,user_8778,0,2,33,S_EU,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.00,100.0,0.0,100.00,7.65,8.51,Apple,2.55,2.55,2.550000
1842,user_1842,0,0,49,W_EU,0.0,0.0,0.0,0.0,0.0,3.36,3.3,3.74,4.47,1.12,0.00,63.26,100.0,0.0,20.75,1904.95,1507.61,Android,1.28,500.00,90.711905
10156,user_10156,0,5,31,CEE_EU,0.0,0.0,0.0,0.0,0.0,8.46,0.0,88.44,0.00,1.14,0.00,0.00,100.0,0.0,1.96,1064.06,1077.71,Android,0.60,55.29,9.673273
146,user_146,0,0,39,W_EU,0.0,0.0,0.0,0.0,0.0,0.00,0.0,65.16,34.84,0.00,0.00,0.00,100.0,0.0,0.00,146.54,212.77,Apple,5.07,51.06,20.934286
7486,user_7486,0,0,75,W_EU,0.0,0.0,0.0,0.0,0.0,3.61,0.0,80.55,0.00,1.44,14.39,0.00,100.0,0.0,0.00,554.03,700.00,Android,1.00,85.55,26.382381


In [0]:
users_mod.paid_plan.value_counts()

0.0    14394
1.0     1150
Name: paid_plan, dtype: int64

In [0]:
users_mod.loc[users_mod['brand']== "Apple", 'Apple'] = 1 
users_mod.loc[users_mod['brand']!= "Apple", 'Apple'] = 0 
users_mod=users_mod.drop(columns="brand")
users_mod.sample(5)

,user_id,crypto_on,num_contacts,user_age,Cluster,paid_plan,Agriculture,Mining,Construction,Manufacturing,Transportation,Wholesale,Retail,Finance,Services,Public_Admin,EXCHANGE,TOPUP_IN,TRANSFER_IN,TRANSFER_OUT,total_out,total_in,min_amount_usd,max_amount_usd,mean_amount_usd,Apple
10490,user_10490,0,20,32,S_EU,0.0,0.0,0.0,0.0,29.25,64.36,0.0,0.00,0.00,0.00,0.0,0.00,100.00,0.00,6.39,399.80,476.59,12.77,168.78,79.960000,0.0
7596,user_7596,0,23,32,S_EU,0.0,0.0,0.0,0.0,0.00,0.62,0.0,2.26,0.00,0.00,0.0,0.00,100.00,0.00,97.12,1701.72,1710.64,0.94,1652.69,170.172000,0.0
10546,user_10546,0,36,31,CEE_EU,0.0,0.0,0.0,0.0,0.00,0.00,0.0,25.29,1.14,18.78,0.0,13.75,97.63,2.37,41.04,783.99,770.19,0.11,147.24,4.637283,0.0
2073,user_2073,0,0,56,S_EU,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
10667,user_10667,0,25,21,CEE_EU,0.0,0.0,0.0,0.0,0.00,18.14,0.0,5.81,41.07,0.00,0.0,28.28,11.81,88.19,6.69,195.23,227.36,0.50,179.03,16.489167,0.0


In [0]:
#users_mod=pd.get_dummies(users_mod,columns=["Cluster"])
#users_mod=users_mod.drop(columns=["country"])
#users_mod.head()

In [0]:
info = pd.DataFrame(users_mod.dtypes).T.rename(index={0:'Data type'})
info = info.append(pd.DataFrame(users_mod.isnull().sum()).T.rename(index={0: 'Zero values (number)'}))
info = info.append(pd.DataFrame(100*users_mod.isnull().sum()/users_mod.shape[0]).T.rename(index={0: 'Zero values (%)'}))

info

,user_id,crypto_on,num_contacts,user_age,Cluster,paid_plan,Agriculture,Mining,Construction,Manufacturing,Transportation,Wholesale,Retail,Finance,Services,Public_Admin,EXCHANGE,TOPUP_IN,TRANSFER_IN,TRANSFER_OUT,total_out,total_in,min_amount_usd,max_amount_usd,mean_amount_usd,Apple
Data type,object,int64,int64,int64,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
Zero values (number),0,0,0,0,0,0,1735,1735,1735,1735,1735,1735,1735,1735,1735,1735,1735,925,925,1735,748,748,1693,1693,1693,0
Zero values (%),0,0,0,0,0,0,11.1619,11.1619,11.1619,11.1619,11.1619,11.1619,11.1619,11.1619,11.1619,11.1619,11.1619,5.95085,5.95085,11.1619,4.81215,4.81215,10.8917,10.8917,10.8917,0


In [0]:
#test=users_mod[users_mod.total_in==0]
#test=test[test.total_out==0]
#test.shape

In [0]:
users_mod=users_mod.fillna(0)

info = pd.DataFrame(users_mod.dtypes).T.rename(index={0:'Data type'})
info = info.append(pd.DataFrame(users_mod.isnull().sum()).T.rename(index={0: 'Zero values (number)'}))
info = info.append(pd.DataFrame(100*users_mod.isnull().sum()/users_mod.shape[0]).T.rename(index={0: 'Zero values (%)'}))

info

,user_id,crypto_on,num_contacts,user_age,Cluster,paid_plan,Agriculture,Mining,Construction,Manufacturing,Transportation,Wholesale,Retail,Finance,Services,Public_Admin,EXCHANGE,TOPUP_IN,TRANSFER_IN,TRANSFER_OUT,total_out,total_in,min_amount_usd,max_amount_usd,mean_amount_usd,Apple
Data type,object,int64,int64,int64,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
Zero values (number),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Zero values (%),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
users_mod.loc[(users_mod.total_in == 0)|(users_mod.total_out == 0), 'No_Trans'] = 1 
users_mod.loc[(users_mod.total_in > 0)|(users_mod.total_out > 0), 'No_Trans'] = 0

print(users_mod.No_Trans.value_counts())

0.0    14795
1.0      749
Name: No_Trans, dtype: int64


##User Modelling Clusters

###Test 1 (No Cluster) Kept No_Trans

In [0]:
users_mod.columns

Index(['user_id', 'crypto_on', 'num_contacts', 'user_age', 'Cluster',
       'paid_plan', 'Agriculture', 'Mining', 'Construction', 'Manufacturing',
       'Transportation', 'Wholesale', 'Retail', 'Finance', 'Services',
       'Public_Admin', 'EXCHANGE', 'TOPUP_IN', 'TRANSFER_IN', 'TRANSFER_OUT',
       'total_out', 'total_in', 'min_amount_usd', 'max_amount_usd',
       'mean_amount_usd', 'Apple', 'No_Trans'],
      dtype='object')

In [0]:
users_clust=users_mod.drop(columns=["user_id","Cluster"])

In [0]:
sc = MinMaxScaler()
X = sc.fit_transform(users_clust.values)

for k in range(3,10):
    kmeans = KMeans(n_clusters = k, n_init=30, random_state=42)
    kmeans.fit(X)
    clusters = kmeans.predict(X)
    silhouette_avg = silhouette_score(X, clusters)
    print("For k =", k, "The average silhouette_score is :", silhouette_avg)

For k = 3 The average silhouette_score is : 0.30260792678010495
For k = 4 The average silhouette_score is : 0.31833922262037795
For k = 5 The average silhouette_score is : 0.3466433534338579
For k = 6 The average silhouette_score is : 0.36637555292953866
For k = 7 The average silhouette_score is : 0.29149996520411486
For k = 8 The average silhouette_score is : 0.2970134348171964
For k = 9 The average silhouette_score is : 0.28043552885427264


*The best silhouette is found for k =* **6**. 

In [0]:
k_u = 6
kmeans_cust = KMeans(n_clusters = k_u, n_init=30, random_state=42)
cust_clusters = kmeans_cust.fit_predict(X)

centroids = sc.inverse_transform(np.array(kmeans_cust.cluster_centers_))

test=pd.DataFrame(centroids, columns=users_clust.columns)
test=round(test,2)
test=test.rename(columns={"user_settings_crypto_unlocked":"crypto"})
test

,crypto_on,num_contacts,user_age,paid_plan,Agriculture,Mining,Construction,Manufacturing,Transportation,Wholesale,Retail,Finance,Services,Public_Admin,EXCHANGE,TOPUP_IN,TRANSFER_IN,TRANSFER_OUT,total_out,total_in,min_amount_usd,max_amount_usd,mean_amount_usd,Apple,No_Trans
0,-0.00,14.49,36.76,0.04,0.07,0.15,0.01,1.05,4.30,0.31,30.42,10.27,7.59,0.34,15.76,97.39,2.52,22.37,3487.83,2500.70,8.08,525.47,78.31,0.00,0.0
1,-0.00,19.88,36.78,0.06,0.08,0.09,0.02,1.12,3.95,0.27,29.83,9.98,7.17,0.29,15.55,96.62,3.01,23.18,5381.35,3422.92,12.10,824.46,99.37,1.00,0.0
2,1.00,18.89,34.72,0.16,0.06,0.04,0.01,0.98,4.12,0.33,29.06,9.90,6.64,0.39,21.47,95.56,4.04,24.65,7463.56,4487.97,1.79,843.84,60.76,-0.00,-0.0
3,1.00,24.44,34.92,0.28,0.05,0.08,0.02,1.08,3.99,0.33,28.63,8.98,7.16,0.38,22.91,95.08,4.59,24.07,9073.36,5884.39,5.71,1130.62,72.08,1.00,0.0
4,0.07,3.11,35.62,0.00,0.00,0.00,0.00,-0.00,0.00,-0.00,0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,-0.00,-0.00,-0.00,0.00,0.00,0.00,0.46,1.0
5,0.05,27.97,36.12,0.04,0.07,0.22,0.00,0.58,3.05,0.25,26.43,9.31,4.49,0.36,6.58,13.09,69.86,44.98,1925.78,1018.23,12.91,421.60,70.67,0.43,0.0


In [0]:
test=users_clust
test["Clusters"]=cust_clusters
test.Clusters.value_counts()

1    5773
0    5463
2    1497
3    1241
5     821
4     749
Name: Clusters, dtype: int64

###Test 2 (No Cluster,user_age) Droped No_Trans

In [0]:
users_mod.columns

Index(['user_id', 'crypto_on', 'num_contacts', 'user_age', 'Cluster',
       'paid_plan', 'Agriculture', 'Mining', 'Construction', 'Manufacturing',
       'Transportation', 'Wholesale', 'Retail', 'Finance', 'Services',
       'Public_Admin', 'EXCHANGE', 'TOPUP_IN', 'TRANSFER_IN', 'TRANSFER_OUT',
       'total_out', 'total_in', 'min_amount_usd', 'max_amount_usd',
       'mean_amount_usd', 'Apple', 'No_Trans'],
      dtype='object')

In [0]:
users_clust=users_mod.drop(columns=["user_id","Cluster","paid_plan","user_age"])
users_clust=users_clust[users_clust.No_Trans==0]
users_clust=users_clust.drop(columns="No_Trans")

In [0]:
sc = MinMaxScaler()
X = sc.fit_transform(users_clust.values)

for k in range(3,10):
    kmeans = KMeans(n_clusters = k, n_init=30, random_state=42)
    kmeans.fit(X)
    clusters = kmeans.predict(X)
    silhouette_avg = silhouette_score(X, clusters)
    print("For k =", k, "The average silhouette_score is :", silhouette_avg)

For k = 3 The average silhouette_score is : 0.3621390094274444
For k = 4 The average silhouette_score is : 0.37095674151928365
For k = 5 The average silhouette_score is : 0.4065367328957977
For k = 6 The average silhouette_score is : 0.36378383957054017
For k = 7 The average silhouette_score is : 0.3381142449547753
For k = 8 The average silhouette_score is : 0.32384158351543724
For k = 9 The average silhouette_score is : 0.316868671729214


*The best silhouette is found for k =* **5**. 

In [0]:
k_u = 5
kmeans_cust = KMeans(n_clusters = k_u, n_init=30, random_state=42)
cust_clusters = kmeans_cust.fit_predict(X)

centroids = sc.inverse_transform(np.array(kmeans_cust.cluster_centers_))

test=pd.DataFrame(centroids, columns=users_clust.columns)
test=round(test,2)
test=test.rename(columns={"user_settings_crypto_unlocked":"crypto"})
test

,crypto_on,num_contacts,Agriculture,Mining,Construction,Manufacturing,Transportation,Wholesale,Retail,Finance,Services,Public_Admin,EXCHANGE,TOPUP_IN,TRANSFER_IN,TRANSFER_OUT,total_out,total_in,min_amount_usd,max_amount_usd,mean_amount_usd,Apple
0,1.00,18.89,0.06,0.04,0.01,1.01,4.12,0.33,29.04,9.90,6.64,0.39,21.48,95.51,4.09,24.64,7460.11,4486.06,1.79,843.83,60.75,-0.00
1,1.00,24.46,0.05,0.08,0.02,1.09,3.99,0.33,28.61,8.95,7.16,0.38,22.93,95.16,4.52,24.09,9053.88,5878.66,5.72,1130.84,72.08,1.00
2,0.00,19.86,0.08,0.09,0.02,1.11,3.93,0.25,29.84,9.94,7.17,0.29,15.56,96.75,3.01,23.20,5387.01,3426.93,12.10,824.93,99.43,1.00
3,-0.00,14.50,0.07,0.15,0.01,1.05,4.30,0.31,30.41,10.27,7.59,0.34,15.74,97.40,2.53,22.40,3487.43,2500.43,8.08,525.44,78.30,0.00
4,0.05,27.97,0.07,0.22,0.00,0.56,3.15,0.35,26.46,9.63,4.48,0.36,6.64,12.87,69.26,44.46,1957.87,1027.27,12.88,422.30,70.57,0.44


In [0]:
test=users_clust
test["Clusters"]=cust_clusters
test.Clusters.value_counts()

2    5765
3    5464
0    1498
1    1240
4     828
Name: Clusters, dtype: int64

###Test 3 (No Cluster,user_age) Droped No_Trans



(No user_age,plan, num_contacts,Agriculture, Mining, Manufacture,Construction,Wholesale,Apple, Cluster) \
 Droped No_Trans

In [0]:
users_mod.columns

Index(['user_id', 'crypto_on', 'num_contacts', 'user_age', 'Cluster',
       'paid_plan', 'Agriculture', 'Mining', 'Construction', 'Manufacturing',
       'Transportation', 'Wholesale', 'Retail', 'Finance', 'Services',
       'Public_Admin', 'EXCHANGE', 'TOPUP_IN', 'TRANSFER_IN', 'TRANSFER_OUT',
       'total_out', 'total_in', 'min_amount_usd', 'max_amount_usd',
       'mean_amount_usd', 'Apple', 'No_Trans'],
      dtype='object')

In [0]:
users_clust=users_mod.drop(columns=["user_id","paid_plan","user_age","num_contacts","Agriculture", 'Mining', 'Construction', 'Manufacturing',\
                                    'Wholesale',"Apple","Cluster","Public_Admin","TOPUP_IN","TRANSFER_IN","total_in"])
users_clust=users_clust[users_clust.No_Trans==0]
users_clust=users_clust.drop(columns="No_Trans")
#users_clust=pd.get_dummies(users_clust,columns=["Cluster"])

In [0]:
sc = MinMaxScaler()
X = sc.fit_transform(users_clust.values)

for k in range(3,10):
    kmeans = KMeans(n_clusters = k, n_init=30, random_state=42)
    kmeans.fit(X)
    clusters = kmeans.predict(X)
    silhouette_avg = silhouette_score(X, clusters)
    print("For k =", k, "The average silhouette_score is :", silhouette_avg)

For k = 3 The average silhouette_score is : 0.36675365354009143
For k = 4 The average silhouette_score is : 0.3556588881504557
For k = 5 The average silhouette_score is : 0.38916432309452537
For k = 6 The average silhouette_score is : 0.39989368965611616
For k = 7 The average silhouette_score is : 0.3788955818662298
For k = 8 The average silhouette_score is : 0.38914589930051163
For k = 9 The average silhouette_score is : 0.393773234463284


*The best silhouette is found for k =* **6**. 

In [0]:
k_u = 6

kmeans_cust = KMeans(n_clusters = k_u, n_init=30, random_state=42)
cust_clusters = kmeans_cust.fit_predict(X)

centroids = sc.inverse_transform(np.array(kmeans_cust.cluster_centers_))

test=pd.DataFrame(centroids, columns=users_clust.columns)
test=round(test,2)
test=test.rename(columns={"user_settings_crypto_unlocked":"crypto"})
test

,crypto_on,Transportation,Retail,Finance,Services,EXCHANGE,TRANSFER_OUT,total_out,min_amount_usd,max_amount_usd,mean_amount_usd
0,-0.0,4.18,70.70,6.09,7.23,2.55,7.42,1815.49,2.71,207.04,21.90
1,-0.0,2.84,19.32,6.66,5.09,48.55,16.22,9035.74,11.76,1313.40,165.56
2,1.0,4.07,28.66,9.50,6.88,21.96,24.68,8104.70,3.55,965.48,65.98
3,-0.0,3.44,19.63,60.83,4.90,2.65,7.49,1922.09,5.23,248.23,43.43
4,-0.0,1.18,7.14,2.44,1.52,2.92,84.22,4239.76,30.25,941.17,162.32
5,-0.0,8.01,17.04,6.31,15.33,2.57,10.92,2233.13,3.18,312.60,24.25


In [0]:
test=users_clust
test["Clusters"]=cust_clusters
test.Clusters.value_counts()

1    3242
0    3095
2    2783
5    2566
4    2140
3     969
Name: Clusters, dtype: int64